In [81]:
def handle_literal(packetValues):
    numbersInPacket = ''
    numberOfBits = 0
    continueBit = 1
    while continueBit == 1:
        continueBit = int(packetValues[0])
        del packetValues[0]
        if continueBit == 1:
            numbersInPacket += ''.join(packetValues[:4])
            del packetValues[:4]
        else:
            numbersInPacket += ''.join(packetValues[:4])
            del packetValues[:4]
        numberOfBits += 5
    #return int(numbersInPacket, 2)
    return str(int(numbersInPacket, 2))

In [82]:
def handle_operator(packetValues, packetVersion):
    lengthTypeBit = int(packetValues[0])
    del packetValues[0]
    # Type 0 = 15 bits for length
    if lengthTypeBit == 0:
        totalLength = int(''.join(packetValues[:15]),2)
        del packetValues[:15]
    # Type 1 = 11 bits for _number of sub-packets contained in this packet_
    else:
        numberOfSubPackets = int(''.join(packetValues[:11]),2)
        del packetValues[:11]
    while packetValues.count('0') != len(packetValues):
        packetVersion = solve_part1(packetValues, packetVersion)
    return packetVersion

In [83]:
# Solve part 1
def solve_part1(packet, packetVersion):
    #print(packet)
    packetVersion += int(''.join(packet[:3]), 2)
    #print(packetVersion)
    packetType = int(''.join(packet[3:6]), 2)
    del packet[:6]
    #print(packetType)
    if packetType == 4:
        handle_literal(packet)
    else:
        packetVersion = handle_operator(packet, packetVersion)
    return packetVersion

In [84]:
def part_2_handle_operator(packetValues, packetType, opList):
    lengthTypeBit = int(packetValues[0])
    del packetValues[0]
    totalLength = 0
    subpacketCount = 0
    # Type 0 = 15 bits for length
    if lengthTypeBit == 0:
        totalLength = int(''.join(packetValues[:15]),2)
        del packetValues[:15]
    # Type 1 = 11 bits for _number of sub-packets contained in this packet_
    else:
        subpacketCount = int(''.join(packetValues[:11]),2)
        del packetValues[:11]

    if packetType == 0:
        opList.append('(')
        if totalLength > 0:
            packetValues = solve_part2(packetValues[:totalLength], opList, 9999)
            del packetValues[:totalLength]            
        else:
            packetValues = solve_part2(packetValues, opList, subpacketCount)
        opList.append('+')
        opList.append(')')
    elif packetType == 1:
        opList.append('(')
        if totalLength > 0:
            packetValues = solve_part2(packetValues[:totalLength], opList, 9999)
            del packetValues[:totalLength]
        else:
            packetValues = solve_part2(packetValues, opList, subpacketCount)
        opList.append('*')
        opList.append(')')
    elif packetType == 2:
        opList.append('(')
        if totalLength > 0:
            solve_part2(packetValues[:totalLength], opList, 9999)
            del packetValues[:totalLength]
        else:
            packetValues = solve_part2(packetValues, opList, subpacketCount)
        opList.append('min')
        opList.append(')')
    elif packetType == 3:
        opList.append('(')
        if totalLength > 0:
            packetValues = solve_part2(packetValues[:totalLength], opList, 9999)
            del packetValues[:totalLength]
        else:
            packetValues = solve_part2(packetValues, opList, subpacketCount)
        opList.append('max')
        opList.append(')')
    elif packetType == 5:
        opList.append('(')
        packetValues = solve_part2(packetValues, opList, 2)
        opList.append('>')
        opList.append(')')
    elif packetType == 6:
        opList.append('(')
        packetValues = solve_part2(packetValues, opList, 2)
        opList.append('<')
        opList.append(')')
    elif packetType == 7:
        opList.append('(')
        packetValues = solve_part2(packetValues, opList, 2)
        opList.append('=')
        opList.append(')')

    #while packetValues.count('0') != len(packetValues):
    #    packetVersion = solve_part1(packetValues, packetVersion)
    return packetValues

In [85]:
# Solve part 2 by setting this up like Reverse Polish Notation
def solve_part2(packet, opList, packetCount):
    for i in range(0, packetCount):
        if packet.count('0') == len(packet) or len(packet) < 4:
            return packet
        #print(packetVersion)
        packetType = int(''.join(packet[3:6]), 2)
        del packet[:6]
        #print(packetType)
        if packetType == 4:
            literalValue = handle_literal(packet)
            opList.append(literalValue)
        else:
            packet = part_2_handle_operator(packet, packetType, opList)
    return packet

In [86]:
import math
def calculate(opList):
    operators = ['+', '*', 'min', 'max', '<', '>', '=']
    opStack = []
    thisOp = []
    for item in opList:
        if item not in operators:
            opStack.append(int(item))
            thisOp.append(int(item))
        else:
            if item == '+':
                if len(thisOp) == 1:
                    total = thisOp[0]
                total = sum(thisOp)
                for i in range(len(thisOp)):
                    opStack.pop()
                thisOp.clear()
                opStack.append(total)
            elif item == '*':
                if len(thisOp) == 1:
                    product = thisOp[0]
                product = math.prod(thisOp)
                for i in range(len(thisOp)):
                    opStack.pop()
                thisOp.clear()
                opStack.append(product)
            elif item == 'min':
                minimum = min(thisOp)
                for i in range(len(thisOp)):
                    opStack.pop()
                thisOp.clear()
                opStack.append(minimum)
            elif item == 'max':
                maximum = max(thisOp)
                for i in range(len(thisOp)):
                    opStack.pop()
                thisOp.clear()
                opStack.append(maximum)
            elif item == '<':
                secondItem = opStack.pop()
                firstItem = opStack.pop()
                if firstItem < secondItem:
                    opStack.append(1)
                else:
                    opStack.append(0)
                thisOp.clear()

            elif item == '>':
                secondItem = opStack.pop()
                firstItem = opStack.pop()
                if firstItem > secondItem:
                    opStack.append(1)
                else:
                    opStack.append(0)
                thisOp.clear()

            elif item == '=':
                secondItem = opStack.pop()
                firstItem = opStack.pop()
                if firstItem == secondItem:
                    opStack.append(1)
                else:
                    opStack.append(0)
                thisOp.clear()
                
    print(opStack)

In [87]:
# Read in XYZ
file = open('puzzleinput.txt')
#inputLines = [line.strip() for line in file]
#inputInt = format(file.readline(), ":016x")
hex_string = file.readline()
#print(bin(inputInt))
bin_string = str(bin(int('1'+hex_string, 16))[3:])
#print(hex_string)
#print(bin_string)

# Solve parts 1 and 2
packetVersion = 0
packetVersion = solve_part1(list(bin_string).copy(), packetVersion)
print('v', packetVersion)
opList = []
print(bin_string)
solve_part2(list(bin_string), opList, 9999)
print(opList)
calculate(opList)
# Print solution
#print("Part 1 solution: There are " + str(part1Solution) + " dots visible after one fold")
#print("Part 2 solution: The code is displayed below: ")

v 943
1100001000001101010000100000000000101110110100110011001111100111011101110100111010101010110000111100001000000000100101100111000000000001010101101001001011000110000110110110010110001001001000100011100110000000001101010011011011000101001111100010110100110000011110100110000000001010110011110011001001001001001010000011100000000001001100111101000110000011011000010000000001011011001100110110110100110110000000001011010010111111100101101110010010100101100111111110110110010100110000000010100110011000000111001001011001000000100111000100101010010110010011111001100101011101001000000001010111011110011010111101000111000110111001110010010101101011000000000100110000000000000100001010100001101011000001101010000111110000000000000010000101010001101011100000110011000111100110000110011001100000000010101100110001011100101010111100000000010101000100000000011000100011100011000100011010000101100000100000000011100100000101111000111110010000111101100110001111111011101001001111110111101100000001100001000000

IndexError: pop from empty list